# Imports

In [3]:
!pip install pip3-autoremove
!pip-autoremove torch torchvision torchaudio -y
!pip install torch torchvision torchaudio xformers --index-url https://download.pytorch.org/whl/cu121
!pip install unsloth

cryptography 44.0.1 is installed but cryptography<44 is required
Redoing requirement with just package name...
pyOpenSSL 25.0.0 is installed but pyOpenSSL<=24.2.1,>=19.1.0 is required
Redoing requirement with just package name...
fsspec 2024.12.0 is installed but fsspec==2024.10.0 is required
Redoing requirement with just package name...
google-api-core 1.34.1 is installed but google-api-core[grpc]<3.0.0dev,>=2.16.0 is required
Redoing requirement with just package name...
notebook 6.5.4 is installed but notebook==6.5.5 is required
Redoing requirement with just package name...
pandas 2.2.3 is installed but pandas==2.2.2 is required
Redoing requirement with just package name...
async-timeout 5.0.1 is installed but async-timeout<5.0.0,>=4.0.0; python_version < "3.11" is required
Redoing requirement with just package name...
scikit-learn 1.2.2 is installed but scikit-learn>=1.3.1 is required
Redoing requirement with just package name...
google-api-core 1.34.1 is installed but google-api-c

In [4]:
from unsloth import FastLanguageModel
import torch
import pandas as pd
from datasets import load_dataset

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


# Define Model

In [5]:
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for\ Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

In [6]:
# modelos: https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/mistral-7b", # Choose ANY! eg teknium/OpenHermes-2.5-Mistral-7B
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

==((====))==  Unsloth 2025.2.15: Fast Mistral patching. Transformers: 4.49.0.
   \\   /|    GPU: Tesla T4. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 7.5. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/4.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/155 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.02k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

In [7]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    use_gradient_checkpointing = True,
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.2.15 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


# Dataset

In [8]:
def correct_data_format(path):
    # Carrega o CSV
    df = pd.read_csv(path)  # substitua pelo caminho correto do seu arquivo
    
    # Renomeia as colunas: 'pergunta' para 'input' e 'resposta' para 'output'
    df.rename(columns={"Pergunta": "input", "Resposta": "output"}, inplace=True)
    
    # Define o texto que será adicionado em todas as linhas na nova coluna 'instruction'
    instruction_text = ("Você é um assistente virtual que auxilia alunos na disciplina de lógica de programação, "
                        "responda as perguntas sobre programação")
    
    # Cria a coluna 'instruction' com o mesmo valor para todas as linhas
    df["instruction"] = instruction_text
    
    # Exibe as primeiras linhas do DataFrame para verificar as alterações
    print(df.head())

    df.to_csv("avaliacoes_combinadas_finetuning_format.csv", index=False)

In [9]:
correct_data_format("/kaggle/input/perguntas-programao-ptbr/avaliacoes_combinadas.csv")

                                               input  \
0                    O que é hoisting em JavaScript?   
1  Qual a diferença entre `null` e `undefined` em...   
2                 O que é uma closure em JavaScript?   
3  O que são template literals (template strings)...   
4  Qual a diferença entre `==` e `===` em JavaScr...   

                                              output  \
0  Hoisting é o comportamento do JavaScript de mo...   
1  `null` é um valor de atribuição. Representa a ...   
2  Uma closure é a combinação de uma função e o a...   
3  Template literals são strings que permitem exp...   
4  `==` é o operador de igualdade abstrata. Ele c...   

                                         instruction  
0  Você é um assistente virtual que auxilia aluno...  
1  Você é um assistente virtual que auxilia aluno...  
2  Você é um assistente virtual que auxilia aluno...  
3  Você é um assistente virtual que auxilia aluno...  
4  Você é um assistente virtual que auxilia aluno..

In [33]:
dataset = load_dataset("csv", data_files="/kaggle/working/avaliacoes_combinadas_finetuning_format.csv")

In [34]:
dataset

DatasetDict({
    train: Dataset({
        features: ['input', 'output', 'instruction'],
        num_rows: 5803
    })
})

In [35]:
alpaca_prompt = """Abaixo segue uma instrução que descreve uma tarefa, acompanhada de uma entrada que fornece um contexto adicional. Escreva uma resposta que complete apropriadamente o pedido.

Instrução:
{}

Entrada:
{}

Resposta:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN

def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass


In [36]:
dataset = dataset["train"].map(formatting_prompts_func, batched = True,)

In [37]:
dataset[1]

{'input': 'Qual a diferença entre `null` e `undefined` em JavaScript?',
 'output': '`null` é um valor de atribuição. Representa a ausência intencional de qualquer valor de objeto. `undefined` significa que uma variável foi declarada, mas não atribuída a nenhum valor.',
 'instruction': 'Você é um assistente virtual que auxilia alunos na disciplina de lógica de programação, responda as perguntas sobre programação',
 'text': 'Abaixo segue uma instrução que descreve uma tarefa, acompanhada de uma entrada que fornece um contexto adicional. Escreva uma resposta que complete apropriadamente o pedido.\n\nInstrução:\nVocê é um assistente virtual que auxilia alunos na disciplina de lógica de programação, responda as perguntas sobre programação\n\nEntrada:\nQual a diferença entre `null` e `undefined` em JavaScript?\n\nResposta:\n`null` é um valor de atribuição. Representa a ausência intencional de qualquer valor de objeto. `undefined` significa que uma variável foi declarada, mas não atribuída a 

In [15]:
from trl import SFTTrainer
from transformers import TrainingArguments

In [16]:
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

Converting train dataset to ChatML (num_proc=2):   0%|          | 0/5803 [00:00<?, ? examples/s]

Applying chat template to train dataset (num_proc=2):   0%|          | 0/5803 [00:00<?, ? examples/s]

Tokenizing train dataset (num_proc=2):   0%|          | 0/5803 [00:00<?, ? examples/s]

Tokenizing train dataset (num_proc=2):   0%|          | 0/5803 [00:00<?, ? examples/s]

In [17]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.741 GB.
4.031 GB of memory reserved.


In [18]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 5,803 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,1.748500
2,1.590200
3,1.569700
4,1.272200
5,0.894600
6,0.658700
7,0.489400
8,0.480900
9,0.463100
10,0.510700


In [19]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

361.5731 seconds used for training.
6.03 minutes used for training.
Peak reserved memory = 4.924 GB.
Peak reserved memory for training = 0.893 GB.
Peak reserved memory % of max memory = 33.403 %.
Peak reserved memory for training % of max memory = 6.058 %.


In [20]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "", # instruction
        "O que é um loop for", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

['<s> Abaixo segue uma instrução que descreve uma tarefa, acompanhada de uma entrada que fornece um contexto adicional. Escreva uma resposta que complete apropriadamente o pedido.\n\nInstrução:\n\n\nEntrada:\nO que é um loop for\n\nResposta:\nO loop for é um conceito importante em JavaScript. Ele é utilizado para diversas operações, incluindo JavaScript. Seu uso adequado melhora a performance e a legibilidade do código.</s>']

In [21]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Continue the fibonnaci sequence.", # instruction
        "1, 1, 2, 3, 5, 8", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

<s> Abaixo segue uma instrução que descreve uma tarefa, acompanhada de uma entrada que fornece um contexto adicional. Escreva uma resposta que complete apropriadamente o pedido.

Instrução:
Continue the fibonnaci sequence.

Entrada:
1, 1, 2, 3, 5, 8

Resposta:
13</s>


In [22]:
model.save_pretrained("lora_model_test")

In [29]:
if False:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference

# alpaca_prompt = You MUST copy from above!

inputs = tokenizer(
[
    alpaca_prompt.format(
        "What is a famous tall tower in Paris?", # instruction
        "", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

["<s> Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nWhat is a famous tall tower in Paris?\n\n### Input:\n\n\n### Response:\nThe Eiffel Tower is a famous tall tower in Paris, France. It is an iron lattice tower located on the Champ de Mars and is one of the most recognizable structures in the world. It was built in 1889 as the entrance arch to the 1889 World's"]

In [ ]:
# Merge to 16bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_16bit", token = "")

# Merge to 4bit
if True: model.save_pretrained_merged("model_teste", tokenizer, save_method = "merged_4bit_forced",)
if True: model.push_to_hub_merged("SEU REPOSITORIO HF", tokenizer, save_method = "merged_4bit_forced", token = "SEU TOKEN")

# Just LoRA adapters
if False: model.save_pretrained_merged("model", tokenizer, save_method = "lora",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "lora", token = "")

Unsloth: Merging 4bit and LoRA weights to 4bit...
This might take 5 minutes...


/usr/local/lib/python3.10/dist-packages/peft/tuners/lora/bnb.py:355: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


Done.
Unsloth: Saving tokenizer... Done.
Unsloth: Saving model... This might take 10 minutes for Llama-7b... Done.
Unsloth: Merging 4bit and LoRA weights to 4bit...
This might take 5 minutes...
Done.
Unsloth: Saving 4bit Bitsandbytes model. Please wait...


README.md:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

  0%|          | 0/1 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/4.13G [00:00<?, ?B/s]

README.md:   0%|          | 0.00/56.0 [00:00<?, ?B/s]

  0%|          | 0/1 [00:00<?, ?it/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

Saved merged_4bit model to https://huggingface.co/Onifin/PLDS
